# Datos

In [1]:
# General Purpose
import pandas as pd
import matplotlib.pyplot as plt
import os
import picimport numpy as np
kle

# FE
from CustomTransform import DistanceEncoder, SizeMeanEncoder, RangeEncoder, CustomKNN, RangeOtherEncoderImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

# Models
from sklearn.ensemble import Randofrom sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_errore_percentage_error

In [2]:
# Cargamos los datos
dataTrain = pd.read_csv('depDataTrain_Arnau_20-08.csv')
dataTest = pd.read_csv('depDataTest_Arnau_20-08.csv')

In [3]:
dataTest.head()

,ide_floor,ide_size,ide_price,ide_longitude,ide_latitude,his_quarterly_variation,his_price,his_monthly_variation,his_annual_variation,geo_distrito,...,dem_TamanoMedioDelHogar,dem_PropSinEstudiosUniversitarios,dem_PropSinEstudios,dem_Proporcion_de_nacidos_fuera_de_Espana,dem_PropConEstudiosUniversitarios,dem_PobTotal,dem_NumViviendas,dem_EdadMedia,dem_Densidad_(Habit/Ha),fe_micole_weights
0,2.0,175.0,1400000.0,-3.715417,40.422832,0.5,4773,0.3,-3.7,Moncloa - Aravaca,...,2.328217,0.343683,0.066403,20.963846,0.589914,24423,10490.0,46.972342,322.402577,0.084622
1,4.0,71.0,365000.0,-3.706351,40.424913,1.4,5106,0.1,-4.3,Centro,...,1.975877,0.386598,0.083812,32.102460,0.529590,33418,16913.0,43.678945,352.500616,0.095684
2,NaN,625.0,2099000.0,-3.767169,40.466552,0.2,4182,-0.1,-3.6,Moncloa - Aravaca,...,3.506552,0.246480,0.053543,15.365819,0.699977,6957,1984.0,36.697930,21.066101,0.189791
3,0.0,117.0,429000.0,-3.704230,40.424084,1.4,5106,0.1,-4.3,Centro,...,1.975877,0.386598,0.083812,32.102460,0.529590,33418,16913.0,43.678945,352.500616,0.092895
4,0.0,90.0,179000.0,-3.763866,40.396186,1.9,2174,2.4,-3.2,Latina,...,2.512222,0.554332,0.219880,26.475125,0.225788,67418,26836.0,48.380655,236.570161,0.096254


In [67]:
dataTest.columns

Index(['ide_floor', 'ide_size', 'ide_longitude', 'ide_latitude',
       'his_quarterly_variation', 'his_price', 'his_monthly_variation',
       'his_annual_variation', 'geo_distrito', 'geo_barrio',
       'fe_places_weights_imputed',
       'dem_Indice_de_reemplazo_de_la_poblacion_activa',
       'dem_Indice_de_juventud',
       'dem_Indice_de_estructura_de_la_poblacion_activa',
       'dem_Indice_de_dependencia', 'dem_TasaDeParo',
       'dem_TamanoMedioDelHogar', 'dem_PropSinEstudiosUniversitarios',
       'dem_PropSinEstudios', 'dem_Proporcion_de_nacidos_fuera_de_Espana',
       'dem_PropConEstudiosUniversitarios', 'dem_PobTotal', 'dem_NumViviendas',
       'dem_EdadMedia', 'dem_Densidad_(Habit/Ha)', 'fe_micole_weights'],
      dtype='object')

# Modelado

In [4]:
# Train-Test target

target = 'ide_price'

y_train = dataTrain.pop(target)
X_train = dataTrain

y_test = dataTest.pop(target)
X_test = dataTest

### Transformers

In [5]:
# Distance Encoder:
distanceOrigin = [['sol', [40.414650, -3.700400]],
                  ['nmi', [40.446278, -3.691814]],
                  ['pca', [40.466070, -3.689280]]]
distanceEncoderColumn = ['ide_latitude', 'ide_longitude']
distanceEncoder = list()
for i in range(3):
    distanceEncoder.append(
        DistanceEncoder('fe_distance_' + distanceOrigin[i][0], distanceEncoderColumn, distanceOrigin[i][1]))

# Size Mean Encoder:
sizeMeanEncoderColumn = 'ide_size'
sizeMeanGroup = ['barrio', 'distrito']
sizeMeanEncoders = list()
for i in range(2):
    sizeMeanEncoders.append(
        SizeMeanEncoder('fe_mean_size_' + sizeMeanGroup[i], sizeMeanEncoderColumn, 'geo_' + sizeMeanGroup[i]))

# Range Enconders:
rangeEncoders = list()
rangeEncoders.append(RangeEncoder('fe_latitude_bins', 'ide_latitude', 10))
rangeEncoders.append(RangeEncoder('fe_longitude_bins', 'ide_longitude', 10))

# Custom KNNImputer
imputer = CustomKNNImputer(n_neighbors = 5, weights = 'distance')

# Column Transformer for OneHotEncoder
categorical_features = ['geo_distrito', 'geo_barrio', 'fe_latitude_bins', 'fe_longitude_bins']
oneHotColumnTransformer = ColumnTransformer([
    ("oneHotEncoder", OneHotEncoder(handle_unknown='ignore'),
     categorical_features)
], remainder = 'passthrough')

## Model Import

In [7]:
model = RandomForestRegressor(n_estimators = 460,
                              max_depth = None,
                              max_features = 'auto',
                              min_samples_split = 2,
                              min_samples_leaf = 1,
                              n_jobs = -1,
                              random_state = 42)

pipeline = Pipeline([('Encoder1', distanceEncoder[0]),
                     ('Encoder2', distanceEncoder[1]),
                     ('Encoder3', distanceEncoder[2]),
                     ('Encoder4', sizeMeanEncoders[0]),
                     ('Encoder5', sizeMeanEncoders[1]),
                     ('Encoder6', rangeEncoders[0]),
                     ('Encoder7', rangeEncoders[1]),
                     ('Imputer1', imputer),
                     ('Encoder8', oneHotColumnTransformer),
                     ('Model', model)])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('Encoder1',
                 DistanceEncoder(new_columns='fe_distance_sol',
                                 origin=[40.41465, -3.7004],
                                 transformed_columns=['ide_latitude',
                                                      'ide_longitude'])),
                ('Encoder2',
                 DistanceEncoder(new_columns='fe_distance_nmi',
                                 origin=[40.446278, -3.691814],
                                 transformed_columns=['ide_latitude',
                                                      'ide_longitude'])),
                ('Encoder3',
                 DistanceEncoder(new_columns='fe_distance_...
                              transformed_columns='ide_longitude')),
                ('Imputer1',
                 CustomKNNImputer(n_neighbors=5, weights='distance')),
                ('Encoder8',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('o

In [9]:
dataTest_y_PRED = pipeline.predict(X_test)

mape = mean_absolute_percentage_error(y_test, dataTest_y_PRED)

msg = "%s: %f" % ('MAPE', mape)
print(msg)

MAPE: 0.163976


## Test Results EDA

In [20]:
test_full = pd.DataFrame(y_test)
test_full['ide_price prediction'] = dataTest_y_PRED

In [113]:
test_results = pd.merge(X_test, test_full, left_index=True, right_index=True)

In [110]:
encoder1 = RangeEncoder('ide_price_bins', 'ide_price', 32)
encoder1.fit(test_results)
test_results = encoder1.transform(test_results)

encoder2 = RangeEncoder('ide_size_bins', 'ide_size', 32)
encoder2.fit(test_results)
test_results = encoder2.transform(test_results)

In [111]:
df = test_results.copy()
filter_by_list = ['geo_distrito', 'geo_barrio', 'ide_price_bins', 'ide_size_bins']
first = True

for filter_by in filter_by_list:
    
    metrics = pd.DataFrame(columns = [filter_by, "MSE", "MAE", "MAPE", "R2"])
    
    for item in df[filter_by].tolist():
        
        values = df.loc[(df[filter_by] == item)]

        metrics_ = pd.DataFrame({filter_by: item,
                            "MSE": np.round(mean_squared_error(values["ide_price"], values["ide_price prediction"]), 4),
                            "MAE": np.round(mean_absolute_error(values["ide_price"], values["ide_price prediction"]), 4),
                            "MAPE": np.round(mean_absolute_percentage_error(values["ide_price"], values["ide_price prediction"]), 4)}, index=[0])
        metrics = metrics.append(metrics_)
    
    metrics = metrics.groupby(filter_by).mean()

    if first:
        with pd.ExcelWriter("predictions_output/predictions_metrics.xlsx", engine='openpyxl', mode="w") as writer: # first one in write mode to delete previous data (new excel)
            metrics.to_excel(writer, sheet_name = 'Métricas '+filter_by, index=True)
        first = False
    else:
        with pd.ExcelWriter("predictions_output/predictions_metrics.xlsx", engine='openpyxl', mode="a") as writer: # first one in write mode to delete previous data (new excel)
            metrics.to_excel(writer, sheet_name = 'Métricas '+filter_by, index=True)